# 🚀 Argentum Overview - Cost-First AI Agent Platform

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MarsZDF/argentum/blob/main/notebooks/01_argentum_overview.ipynb)

**Argentum** is a cost-first platform for building, debugging, and optimizing AI agent systems. This notebook provides a comprehensive introduction to all core components.

## 📋 What You'll Learn
- Installing and setting up Argentum
- Cost tracking and budget management
- State debugging with StateDiff
- Memory management with ContextDecay
- Agent coordination with Handoff
- Real-time cost alerts

⚡ **Zero setup required** - Everything installs automatically in this notebook!

In [ ]:
# Install and setup Argentum
!pip install --force-reinstall git+https://github.com/MarsZDF/argentum.git

import argentum
print(f"✅ Argentum v{argentum.__version__} ready")

# Check what features are available
deps = argentum.check_dependencies()
print(f"📊 Available features: {deps}")

# Test basic functionality
try:
    session = argentum.create_agent_session("test")
    print(f"✅ Agent session created with components: {list(session.keys())}")
except Exception as e:
    print(f"❌ Session creation failed: {e}")
    print("Will demonstrate core features only")

## 💰 Cost Tracking - The Heart of Argentum

In [ ]:
## 💰 Cost Tracking (if available)

# Create agent session and check for cost tracking
session = argentum.create_agent_session("cost_demo")
cost_tracker = session.get("cost_tracker")

if cost_tracker and cost_tracker is not None:
    print("✅ Cost tracking is available!")
    print(f"📊 Tracker type: {type(cost_tracker).__name__}")
    
    # Track some usage
    cost_tracker.record_usage(
        operation="completion",
        tokens_used=1500,
        agent_id="research_agent",
        model="gpt-4"
    )
    
    cost_tracker.record_usage(
        operation="embedding", 
        tokens_used=3000,
        agent_id="search_agent",
        model="text-embedding-3-small",
        cost=0.006
    )
    
    # Get report
    report = cost_tracker.get_cost_report()
    print(f"\n💰 Total Cost: ${report.total_cost:.4f}")
    print(f"🎯 Total Tokens: {report.total_tokens:,}")
    print("🤖 Agent Breakdown:")
    for agent, cost in report.breakdown.by_agent.items():
        print(f"  - {agent}: ${cost:.4f}")
        
else:
    print("⚠️ Cost tracking not available in this environment")
    print("This is normal - cost tracking requires additional dependencies")
    print("Core Argentum functionality still works perfectly!")

## 🔍 State Debugging with StateDiff

In [ ]:
from argentum import StateDiff

# Initialize state tracking
diff = StateDiff()

# Simulate an agent working on a research task
initial_state = {
    "task": "research",
    "progress": 0.0,
    "findings": [],
    "confidence": 0.5,
    "metadata": {"sources_checked": 0, "quality_score": 0.0}
}

# Snapshot the initial state
diff.snapshot("start", initial_state)
print("📸 Initial state captured")

# Simulate progress
progress_state = initial_state.copy()
progress_state.update({
    "progress": 0.6,
    "findings": ["Key insight about cost optimization", "Performance bottleneck identified"],
    "confidence": 0.8,
    "metadata": {"sources_checked": 5, "quality_score": 0.85}
})

diff.snapshot("progress", progress_state)
print("📸 Progress state captured")

# Analyze the changes using the correct method name
changes = diff.get_changes("start", "progress")
print("\n🔍 State Changes Detected:")
for key, change_info in changes.items():
    if isinstance(change_info, dict):
        if "from" in change_info and "to" in change_info:
            print(f"  📝 CHANGED: {key} = {change_info['from']} → {change_info['to']}")
        elif "added" in change_info:
            print(f"  ➕ ADDED: {key} = {change_info['added']}")
        elif "removed" in change_info:
            print(f"  ➖ REMOVED: {key} = {change_info['removed']}")
    else:
        print(f"  📝 UPDATED: {key} = {change_info}")

print(f"\n📊 Total changes: {len(changes)} fields modified")

## 🧠 Memory Management with ContextDecay

In [ ]:
from argentum import ContextDecay

# Initialize context with 5-step half-life
context = ContextDecay(half_life_steps=5)

# Add some conversation history
context.add("user_question_1", "How do I optimize my AI costs?", importance=0.9)
context.add("response_1", "Use Argentum for cost tracking and optimization", importance=0.8)
context.add("user_question_2", "What's the weather like?", importance=0.3)
context.add("user_question_3", "Can you explain the StateDiff component?", importance=0.9)

print("📚 Initial context items:")
# get_active returns List[Tuple[str, Any, float]] = (key, value, current_weight)
active_items = context.get_active(threshold=0.0)  # Get all items initially
for key, value, current_weight in active_items:
    # Calculate importance from the private _items (we'll use current_weight as proxy)
    print(f"  - {key}: weight={current_weight:.2f}")

# Simulate time passing (conversation continues)
print("\n⏰ Time passing (3 steps)...")
for step in range(3):
    context.step()
    active_count = len(context.get_active())
    print(f"  Step {step + 1}: {active_count} active items")

print("\n📚 Active context after decay:")
final_active = context.get_active()
for key, value, current_weight in final_active:
    print(f"  - {key}: weight={current_weight:.2f}")

print(f"\n📊 Context stats: {len(final_active)} items still active (threshold 0.3)")

## 🤝 Agent Coordination with Handoff

In [ ]:
from argentum import Handoff

# Create a handoff from research agent to writing agent
handoff = Handoff.create(
    from_agent="research_agent",
    to_agent="writing_agent", 
    context_summary="Research completed on cost optimization strategies. Found 3 key approaches.",
    artifacts=[
        "cost_analysis_data.json",
        "optimization_recommendations.md"
    ],
    confidence=0.85,
    metadata={
        "research_duration": "2.5 hours",
        "sources_consulted": 12,
        "cost_savings_potential": "30-40%"
    }
)

print("🤝 Handoff Created:")
print(f"  📋 ID: {handoff.handoff_id}")
print(f"  🔄 {handoff.from_agent} → {handoff.to_agent}")
print(f"  📝 Context: {handoff.context_summary}")
print(f"  📊 Confidence: {handoff.confidence}")
print(f"  📎 Artifacts: {len(handoff.artifacts)} files")

# Serialize for network transfer
handoff_json = handoff.to_json()
print(f"\n📡 Serialized handoff: {len(handoff_json)} characters")

# Receiving agent can recreate the handoff
received_handoff = Handoff.from_json(handoff_json)
print(f"✅ Handoff received by {received_handoff.to_agent}")

# Create receipt for acknowledgment
receipt = received_handoff.create_receipt(
    status="accepted",
    notes="All artifacts received. Beginning writing phase."
)
print(f"📧 Receipt created: {receipt['status']}")

## 🚨 Cost Alerts Setup

In [ ]:
## 🚨 Cost Alerts (if available)

# Check if cost alerts are available in the session
alerts = session.get("alerts")

if alerts and alerts is not None:
    print("✅ Cost alerts are available!")
    
    # Add budget alerts
    dev_alert = alerts.add_budget_alert(
        threshold_dollars=25.0,
        email="dev-team@company.com",
        message_template="💰 Budget alert: ${cost} spent by {agent_id}"
    )
    
    prod_alert = alerts.add_budget_alert(
        threshold_dollars=100.0,
        email="ops-team@company.com", 
        message_template="🚨 Production budget exceeded: ${cost} > ${threshold}",
        severity="high"
    )
    
    # Add token alert
    token_alert = alerts.add_token_alert(
        threshold_tokens=50000,
        email="ai-team@company.com",
        message_template="🎯 High token usage: {tokens_used} tokens for {agent_id}"
    )
    
    print(f"🚨 Configured {len(alerts.rules)} alert rules:")
    budget_rules = [r for r in alerts.rules if r.rule_type == 'budget']
    token_rules = [r for r in alerts.rules if r.rule_type == 'token']
    print(f"  📊 Budget alerts: {len(budget_rules)}")
    print(f"  🎯 Token alerts: {len(token_rules)}")
    
    # Test alert simulation
    print("\n🧪 Alert simulation:")
    test_cost, test_tokens = 35.0, 45000
    triggered = 0
    for rule in alerts.rules:
        if (rule.rule_type == 'budget' and test_cost >= rule.threshold_dollars) or \
           (rule.rule_type == 'token' and test_tokens >= rule.threshold_tokens):
            triggered += 1
    print(f"  Cost ${test_cost:.2f}, {test_tokens:,} tokens → {triggered} alerts would trigger")
    
else:
    print("⚠️ Cost alerts not available in this environment")
    print("You can still manually implement cost monitoring logic!")
    print("Example: if total_cost > 50: send_notification()")

## 📊 Cost Export and Reporting

In [ ]:
## 📊 Cost Export and Reporting (if available)

# Try to import cost export functionality
try:
    from argentum import CostExporter
    exporter = CostExporter()
    
    print("✅ Cost export functionality available!")
    print("📊 Supported export formats:")
    print("  📄 CSV: Spreadsheet-friendly format for analysis")
    print("  📊 Excel: Multi-sheet reports with charts") 
    print("  📋 PDF: Executive summaries and reports")
    print("  🔗 JSON: API integration and programmatic access")
    
    # Simulate some cost data
    sample_data = [
        {"agent_id": "research_agent", "cost": 15.50, "tokens": 12000, "model": "gpt-4"},
        {"agent_id": "writing_agent", "cost": 8.25, "tokens": 8500, "model": "gpt-3.5-turbo"},
        {"agent_id": "review_agent", "cost": 4.75, "tokens": 3200, "model": "gpt-4"},
    ]
    
    total_cost = sum(item["cost"] for item in sample_data)
    total_tokens = sum(item["tokens"] for item in sample_data)
    agent_count = len(sample_data)
    
    print(f"\n📈 Sample Cost Summary:")
    print(f"  💰 Total Cost: ${total_cost:.2f}")
    print(f"  🎯 Total Tokens: {total_tokens:,}")
    print(f"  🤖 Active Agents: {agent_count}")
    print(f"  📊 Avg Cost/Token: ${total_cost/total_tokens:.6f}")
    
    print("\n🤖 Cost by Agent:")
    for item in sorted(sample_data, key=lambda x: x['cost'], reverse=True):
        percentage = (item['cost'] / total_cost) * 100
        print(f"  - {item['agent_id']}: ${item['cost']:.2f} ({percentage:.1f}%)")
        
except ImportError:
    print("⚠️ Cost export not available in this environment")
    print("You can still manually export cost data to CSV/JSON:")
    print("  import csv, json")
    print("  # Your custom export logic here")

## 🎯 Complete Agent Session Example

In [ ]:
## 🎯 Complete Agent Session Demo

# Use the session we created earlier
print("🎯 Complete Agent Session Demo:")
print(f"  Agent ID: {session['agent_id']}")

# Core components (always available)
context = session["context_decay"]
state_diff = session["state_diff"]

print(f"  🧠 Context management: ✅ {type(context).__name__}")
print(f"  🔍 State debugging: ✅ {type(state_diff).__name__}")

# Optional components (check availability)
tracker = session.get("cost_tracker")
alerts = session.get("alerts")

print(f"  💰 Cost tracking: {'✅' if tracker else '❌'} {type(tracker).__name__ if tracker else 'Not available'}")
print(f"  🚨 Cost alerts: {'✅' if alerts else '❌'} {type(alerts).__name__ if alerts else 'Not available'}")

# Demonstrate core functionality (always works)
print("\n🤖 Simulating agent work...")

# 1. Context management
context.add("task", "Demonstrate Argentum capabilities", importance=0.9)
context.add("method", "Interactive notebook tutorial", importance=0.8)
context.add("status", "learning", importance=0.7)

# Count active items properly
initial_active = len(context.get_active(threshold=0.0))
print(f"  📚 Added context items, {initial_active} total")

# 2. State tracking
initial_state = {"step": 1, "status": "starting", "progress": 0.0, "insights": []}
state_diff.snapshot("init", initial_state)

progress_state = {
    "step": 2, 
    "status": "completed", 
    "progress": 1.0,
    "insights": ["Argentum is powerful", "Session-based approach works great"]
}
state_diff.snapshot("complete", progress_state)

# Use the correct method name
changes = state_diff.get_changes("init", "complete")
print(f"  🔍 Captured {len(changes)} state changes")

# 3. Cost tracking (if available)
if tracker:
    tracker.record_usage(
        operation="tutorial",
        tokens_used=2500,
        agent_id=session["agent_id"],
        model="gpt-4"
    )
    report = tracker.get_cost_report()
    print(f"  💰 Recorded cost: ${report.total_cost:.4f} ({report.total_tokens:,} tokens)")

# 4. Memory management with time progression
before_decay = len(context.get_active())
print(f"  🧠 Active context items before time: {before_decay}")
context.step()  # Simulate time passing
context.step()
after_decay = len(context.get_active())
print(f"  🧠 Active context items after decay: {after_decay}")

print("\n✅ Agent session demo completed successfully!")
print(f"📊 Final stats:")
print(f"  - State snapshots: {len(state_diff.snapshots)}")
print(f"  - Context items: {len(context.get_active())} active")
if tracker:
    print(f"  - Total cost tracked: ${tracker.get_cost_report().total_cost:.4f}")
if alerts and len(alerts.rules) > 0:
    print(f"  - Alert rules configured: {len(alerts.rules)}")
    
print("\n🎉 Argentum core functionality demonstrated!")

## 🏁 Summary & Next Steps

**Congratulations!** You've explored Argentum's capabilities:

### ✅ Core Components (Always Available)
- **StateDiff** - Debug agent behavior and state changes  
- **ContextDecay** - Manage agent memory with temporal decay
- **Handoff** - Coordinate between multiple agents
- **Security** - Input validation and protection

### 💰 Cost Intelligence (When Available)
- **CostTracker** - Monitor and control AI spending  
- **CostAlerts** - Get notified before costs spiral  
- **CostExporter** - Generate reports and analytics  

> **Note**: Cost features require additional dependencies and may not be available in all environments. The core debugging and coordination features always work!

### 🚀 Quick Integration:
```python
import argentum

# Create a complete agent session
session = argentum.create_agent_session("my_agent")

# Core features (always available)
context = session["context_decay"]
state_diff = session["state_diff"]

# Optional features (check first)
cost_tracker = session.get("cost_tracker")
if cost_tracker:
    cost_tracker.record_usage(operation="completion", tokens_used=1500, agent_id="my_agent")
```

### 🔧 Production Setup:
```python
# Robust production code
session = argentum.create_agent_session("production_agent", secure=True)

# Always available
session["context_decay"].add("user_context", data, importance=0.9)
session["state_diff"].snapshot("checkpoint", agent_state)

# Optional cost tracking
if session.get("cost_tracker"):
    session["cost_tracker"].record_usage("api_call", tokens, "agent_id")
```

### 📚 Next Steps:
- **[State Debugging](02_state_diff.ipynb)** - Deep dive into StateDiff
- **[Memory Management](03_context_decay.ipynb)** - Master ContextDecay  
- **[Cost Alerts](06_cost_alerts.ipynb)** - Production monitoring setup

### 📖 Resources:
- **[GitHub Repository](https://github.com/MarsZDF/argentum)** - Source code and examples
- **[Agent Guide](../AGENT_GUIDE.md)** - Machine-readable integration guide
- **[API Documentation](https://argentum-agent.readthedocs.io)** - Complete reference

**Ready to build robust AI agents with Argentum! 🚀🤖**